In [9]:
pip install langchain langchain-community sentence-transformers chromadb


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
from pathlib import Path
import json

from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
import re


# Text splitter (para hacer chunks)
try:
    from langchain.text_splitter import RecursiveCharacterTextSplitter
except Exception:
    from langchain_text_splitters import RecursiveCharacterTextSplitter  # alternativa según versión


# Modelo de Ollama que quieres usar
LLM_MODEL_NAME = "deepseek-r1:8b"   # o "llama3:8b" si prefieres

# Rutas a tus JSONL (ajusta la carpeta si la tienes en otra ubicación)
DATA_DIR = Path("../.jsonl")
FINALS_PATH  = DATA_DIR / "finals.jsonl"
HISTORY_PATH = DATA_DIR / "history.jsonl"
PLAYERS_PATH = DATA_DIR / "players.jsonl"
TEAMS_PATH   = DATA_DIR / "teams.jsonl"
COACHES_PATH = DATA_DIR / "coaches.jsonl"


In [11]:
def load_jsonl(path: Path):
    """Carga un .jsonl (un JSON por línea) y devuelve una lista de dicts."""
    data = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            data.append(json.loads(line))
    return data


finals_data  = load_jsonl(FINALS_PATH)
history_data = load_jsonl(HISTORY_PATH)
players_data = load_jsonl(PLAYERS_PATH)
teams_data   = load_jsonl(TEAMS_PATH)
coaches_data = load_jsonl(COACHES_PATH)

len(finals_data), len(history_data), len(players_data), len(teams_data), len(coaches_data)


(46, 31, 24, 18, 19)

In [12]:
import re
import unicodedata
import json

def norm(s: str) -> str:
    if not s:
        return ""
    s = s.strip()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")
    return s

def guess_coach_name(obj: dict) -> str:
    # intenta varios campos típicos
    for key in ["coach_name", "name", "entrenador", "full_name", "person_name"]:
        if obj.get(key):
            return str(obj[key]).strip()
    # fallback: intenta extraer del título "Perfil de X"
    title = obj.get("titulo") or ""
    m = re.search(r"perfil.*?de\s+(.+)$", title, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip()
    return ""

def extract_candidate_clubs(text: str, known_clubs: list[str] | None = None) -> list[str]:
    """
    Opcional: si pasas una lista de clubes del dataset (team_name),
    los detecta dentro del texto y los coloca como tokens.
    """
    if not text:
        return []
    if not known_clubs:
        return []
    t = norm(text).lower()
    found = []
    for club in known_clubs:
        if norm(club).lower() in t:
            found.append(club)
    return found

def render_coach_doc(obj: dict, known_clubs: list[str] | None = None) -> str:
    coach = guess_coach_name(obj)
    titulo = obj.get("titulo") or ""
    texto = obj.get("texto") or ""
    season = obj.get("season_year")

    clubs = extract_candidate_clubs(texto, known_clubs)
    clubs_str = " | ".join(clubs) if clubs else "N/A"

    # ✅ cabecera “token heavy” (mejora retrieval)
    header_lines = [
        f"CATEGORIA: coach",
        f"ENTRENADOR: {coach if coach else 'N/A'}",
        f"TITULO: {titulo}",
        f"TEMPORADA: {season if season is not None else 'N/A'}",
        f"CLAVES: entrenador | tecnico | mister | champions | ucl",
        f"CLUBES_MENCIONADOS: {clubs_str}",
    ]

    # Añade metadata simple como tokens (sin listas/dicts)
    md = obj.get("metadata") or {}
    if isinstance(md, dict):
        simple_md = []
        for k, v in md.items():
            if isinstance(v, (str, int, float, bool)) or v is None:
                simple_md.append(f"{k}={v}")
        if simple_md:
            header_lines.append("METADATA: " + " | ".join(simple_md))

    return "\n".join(header_lines) + "\n\n" + texto


In [ ]:
# Config de chunking (ajústalo según tu caso)
CHUNK_SIZE = 900
CHUNK_OVERLAP = 150

splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", ". ", " ", ""],
)

known_clubs = sorted({t.get("team_name") for t in teams_data if t.get("team_name")})

def item_to_text(item: dict, source_name: str) -> str:
    """Convierte un registro a texto indexable (lo que irá al embedding)."""
    tipo = item.get("tipo", "")

    # ✅ 1) Render especializado por fuente (MEJORA RETRIEVAL)
    if source_name == "coaches":
        # Usa tu Celda 4 (ahora sí)
        return render_coach_doc(item, known_clubs=known_clubs).strip()

    if source_name == "finals":
        titulo = item.get("titulo") or ""
        texto  = item.get("texto")  or ""
        season = item.get("season_year")

        # --- extracción best-effort desde el título ---
        team_a = team_b = score = None
        m = re.search(
            r":\s*([A-Za-zÁÉÍÓÚÑáéíóúñ\s\.]+)\s+(\d+[-–]\d+)\s+([A-Za-zÁÉÍÓÚÑáéíóúñ\s\.]+)",
            titulo
        )
        if m:
            team_a = m.group(1).strip()
            score  = m.group(2).replace("–", "-").strip()
            team_b = m.group(3).strip()

        # --- ganador (solo si el texto lo dice explícitamente) ---
        winner = None
        mw = re.search(
            r"\b([A-ZÁÉÍÓÚÑ][A-Za-zÁÉÍÓÚÑáéíóúñ\s\.]+?)\s+se\s+impuso\b",
            texto
        )
        if mw:
            winner = mw.group(1).strip()

        # --- goleadores (best-effort, sin inventar) ---
        scorers = re.findall(
            r"\bgol(?:\s+decisivo)?\s+de\s+([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)",
            texto,
            flags=re.IGNORECASE
        )
        scorers = list(dict.fromkeys([s.strip() for s in scorers]))  # dedupe
        scorers_str = " | ".join(scorers) if scorers else None
        # --- header SOLO con datos reales ---
        header = [
            "CATEGORIA: final",
            f"TEMPORADA: {season}" if season is not None else None,
            f"TITULO: {titulo}" if titulo else None,
            f"FINAL_EQUIPO_A: {team_a}" if team_a else None,
            f"FINAL_EQUIPO_B: {team_b}" if team_b else None,
            f"MARCADOR: {score}" if score else None,
            f"GANADOR: {winner}" if winner else None,
            f"GOLEADORES: {scorers_str}" if scorers_str else None,
            "CLAVES: final | ganador | campeon | goles | goleadores | ucl | champions",
        ]

        header = [h for h in header if h]  # 🔹 elimina None

        return ("\n".join(header) + "\n\n" + texto).strip()


    # ✅ 2) QA (tu caso actual)
    titulo = item.get("titulo") or ""
    texto  = item.get("texto")  or ""

    if tipo == "qa":
        q = item.get("question", "") or item.get("pregunta", "") or ""
        a = item.get("answer", "")   or item.get("respuesta", "") or ""
        parts = []
        if titulo:
            parts.append(f"Título: {titulo}")
        if q:
            parts.append("Pregunta: " + q)
        if a:
            parts.append("Respuesta: " + a)
        if texto and not a:
            parts.append(texto)
        return "\n".join(parts).strip()

    # ✅ 3) Caso general (fact / otros)
    parts = []
    if titulo:
        parts.append(f"Título: {titulo}")
    if texto:
        parts.append(texto)
    else:
        parts.append(json.dumps(item, ensure_ascii=False))
    return "\n".join(parts).strip()


def build_docs_chunked(data_list: list[dict], source_name: str) -> list[Document]:
    """Crea Documents + los divide en chunks, manteniendo metadata."""
    out_docs: list[Document] = []

    for i, item in enumerate(data_list):
        categoria = item.get("categoria", source_name)
        tipo = item.get("tipo", "")
        doc_id = item.get("id", f"{source_name}_{i}")

        text = item_to_text(item, source_name)
        if not text:
            continue

        # Metadata base
        metadata = {
            "source": source_name,
            "categoria": categoria,
            "tipo": tipo,
            "id": doc_id,
            "season_year": item.get("season_year"),
        }

        # Si tu JSON lleva un campo 'metadata' interno, lo añadimos (sin pisar claves base)
        extra_meta = item.get("metadata") or {}
        if isinstance(extra_meta, dict):
            for k, v in extra_meta.items():
                if k not in metadata:
                    metadata[k] = v

        base_doc = Document(page_content=text, metadata=metadata)

        # Chunking
        chunks = splitter.split_documents([base_doc])
        for ci, ch in enumerate(chunks):
            ch.metadata = dict(ch.metadata)  # copia
            ch.metadata["chunk"] = ci
            ch.metadata["chunk_id"] = f"{doc_id}::chunk_{ci}"
            ch.metadata["parent_id"] = doc_id
            out_docs.append(ch)

    return out_docs


docs_finals  = build_docs_chunked(finals_data,  "finals")
docs_history = build_docs_chunked(history_data, "history")
docs_players = build_docs_chunked(players_data, "players")
docs_teams   = build_docs_chunked(teams_data,   "teams")
docs_coaches = build_docs_chunked(coaches_data, "coaches")

all_docs = docs_finals + docs_history + docs_players + docs_teams + docs_coaches
len(all_docs)


217

In [14]:
# Embeddings
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

from langchain_community.vectorstores.utils import filter_complex_metadata

# texts: List[str]
# metadatas: List[dict]
# ids: List[str] (opcional)

all_docs = filter_complex_metadata(all_docs)   # ✅ AQUÍ

# Vector store (en memoria). Si quieres persistencia:
# vectorstore = Chroma.from_documents(all_docs, embeddings, persist_directory="./chroma_db")
vectorstore = Chroma.from_documents(
    documents=all_docs,
    embedding=embeddings,
)

# Retrievers por categoría usando metadata.source
retriever_all = vectorstore.as_retriever(search_kwargs={"k": 6})

retriever_finals = vectorstore.as_retriever(
    search_kwargs={"k": 8, "filter": {"source": "finals"}}
)

retriever_history = vectorstore.as_retriever(
    search_kwargs={"k": 8, "filter": {"source": "history"}}
)

retriever_players = vectorstore.as_retriever(
    search_kwargs={"k": 8, "filter": {"source": "players"}}
)

retriever_teams = vectorstore.as_retriever(
    search_kwargs={"k": 8, "filter": {"source": "teams"}}
)

retriever_coaches = vectorstore.as_retriever(
    search_kwargs={"k": 8, "filter": {"source": "coaches"}}
)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_14064\811903882.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [15]:
import re
import unicodedata

def _norm(s: str) -> str:
    """Normaliza para comparar (minúsculas, sin acentos, sin dobles espacios)."""
    if not s:
        return ""
    s = s.strip().lower()
    s = "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")
    s = re.sub(r"\s+", " ", s)
    return s

def extract_person_candidate(question: str) -> str | None:
    """
    Intenta extraer el nombre/entidad PERSONA del texto.
    Devuelve:
      - "pep guardiola" si detecta dos palabras con mayúscula
      - "guardiola" si detecta un apellido con mayúscula
      - None si no hay señales claras
    """
    # Dos palabras tipo "Pep Guardiola"
    m = re.search(r"\b([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)\s+([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)\b", question)
    if m:
        return _norm(m.group(0))

    # Una palabra tipo "Guardiola"
    m = re.search(r"\b([A-ZÁÉÍÓÚÑ][a-záéíóúñ]{3,})\b", question)
    if m:
        return _norm(m.group(1))

    return None

def has_person_name(question: str) -> bool:
    return extract_person_candidate(question) is not None

def choose_retrievers(question: str):
    """
    Devuelve una lista ORDENADA (prioridad real).
    REGLA: si hay persona -> coaches/players SIEMPRE primero.
    Teams/finals solo como soporte, history como comodín final.
    """
    q = _norm(question)

    wants_teams  = any(w in q for w in ["equipo", "equipos", "club", "clubes", "palmares", "palmarés", "titulos", "títulos"])
    wants_coach  = any(w in q for w in ["entrenador", "tecnico", "técnico", "mister", "míster", "coach"])
    wants_player = any(w in q for w in ["jugador", "goleador", "asistente", "asistencia", "maximo", "máximo", "apariciones", "partidos", "portero", "defensa", "delantero"])
    wants_finals = any(w in q for w in ["final", "semifinal", "cuartos", "octavos", "eliminatoria", "ronda", "ida", "vuelta"])
    wants_history = any(w in q for w in ["historia", "temporada", "campeon", "campeón", "gan", "titulo", "título"])

    person = has_person_name(question)

    # ✅ ORDEN INTENCIONADO
    if person:
        # Persona siempre manda
        order = [retriever_coaches, retriever_players]

        # Soportes SOLO si hace falta
        if wants_teams or wants_history:
            order += [retriever_teams]
        if wants_finals:
            order += [retriever_finals]

        # History = comodín final (ideal para QA)
        order += [retriever_history, retriever_all]
        return order

    # No persona -> heurística clásica, pero history siempre al final
    if wants_coach:
        return [retriever_coaches, retriever_all, retriever_history]

    if wants_player:
        return [retriever_players, retriever_all, retriever_history]

    if wants_teams:
        return [retriever_teams, retriever_all, retriever_history]

    if wants_finals:
        return [retriever_finals, retriever_all, retriever_history]

    if wants_history:
        return [retriever_history, retriever_all]

    return [retriever_all, retriever_history]

In [ ]:
llm = ChatOllama(
    model=LLM_MODEL_NAME, temperature=0.1,  # opcional
)


C:\Users\Usuario\AppData\Local\Temp\ipykernel_14064\3881863704.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [ ]:
from urllib import response

from sympy import content


def ask_champions(question: str, k: int = 10, max_docs: int = 24):
    retrievers = choose_retrievers(question)

    def retrieve_docs(retriever, query: str):
        # Compatible con distintas versiones:
        if hasattr(retriever, "invoke"):
            return retriever.invoke(query)
        return retriever.get_relevant_documents(query)

    # --- 1) Query expansion (muy importante para perfiles de entrenadores) ---
    person = extract_person_candidate(question)  # de la caja 1
    expanded_queries = [question]

    if person:
        # Añadimos señales para que el embedding “apunte” a coaches
        expanded_queries += [
            f"{person} champions entrenador",
            f"{person} gano la champions entrenador",
            f"{person} perfil entrenador champions"
        ]

    # --- 2) Presupuesto por retriever (coaches primero, teams no ahoga) ---
    # Ajusta estos pesos si quieres
    def budget_for(r):
        if r is retriever_coaches:
            return max(4, k)              # coaches: fuerte
        if r is retriever_players:
            return max(3, k // 2)         # players: medio
        if r is retriever_finals:
            return max(3, k // 2)         # finals: medio
        if r is retriever_teams:
            return max(2, k // 3)         # teams: poco (evita ruido)
        if r is retriever_history:
            return max(2, k // 3)         # history: poco (comodín)
        return max(2, k // 3)

    docs = []
    seen = set()

    # --- 3) Recuperación multi-query por retriever (con dedupe) ---
    for r in retrievers:
        b = budget_for(r)
        collected = []

        for qx in expanded_queries:
            for d in retrieve_docs(r, qx):
                key = d.page_content.strip()
                if key not in seen:
                    seen.add(key)
                    collected.append(d)
                if len(collected) >= b:
                    break
            if len(collected) >= b:
                break

        docs.extend(collected)

        # corte duro para no inflar contexto
        if len(docs) >= max_docs:
            docs = docs[:max_docs]
            break

    # --- 4) Reordenado inteligente del contexto ---
    # Si hay persona: coach/player primero. Si no: fact primero como antes.
    def sort_key(doc):
        meta = doc.metadata or {}
        categoria = (meta.get("categoria") or "").lower()
        tipo = (meta.get("tipo") or "").lower()

        if person:
            # prioridad máxima: fact de coach/player
            if tipo == "fact" and categoria in ("coach", "player"):
                return (0, 0)
            # luego qa de coach/player
            if tipo == "qa" and categoria in ("coach", "player"):
                return (0, 1)
            # luego fact del resto
            if tipo == "fact":
                return (1, 0)
            # luego qa del resto
            if tipo == "qa":
                return (1, 1)
            return (2, 2)

        # sin persona (tu criterio original, pero un pelín más fino)
        if tipo == "fact":
            return (0, 0)
        if tipo == "qa":
            return (1, 0)
        return (2, 0)

    docs = sorted(docs, key=sort_key)

    print("\n📚 DOCUMENTOS RECUPERADOS:\n")
    for d in docs:
        meta = d.metadata or {}
        print(
            f"- source={meta.get('source')} | categoria={meta.get('categoria')} | tipo={meta.get('tipo')} "
            f"| id={meta.get('id')} | chunk={meta.get('chunk')} | season_year={meta.get('season_year')}"
        )
        print("  Extracto:", d.page_content[:180].replace("\n", " "), "...\n")

    # --- 5) Construir contexto con cabecera (útil para LLM) ---
    context_parts = []
    for d in docs:
        meta = d.metadata or {}
        header = (
            f"[source={meta.get('source')} | categoria={meta.get('categoria')} | tipo={meta.get('tipo')} "
            f"| id={meta.get('id')} | chunk_id={meta.get('chunk_id')}]\n"
        )
        context_parts.append(header + d.page_content)

    context_text = "\n\n---\n\n".join(context_parts)

    prompt = f"""
Eres un asistente de QA sobre la Champions League.
Responde SOLO con el CONTEXTO. Si no puedes sostener la respuesta con el CONTEXTO, responde EXACTAMENTE:
"No lo sé según estos datos".

REGLAS ANTI-INVENCIONES (OBLIGATORIAS)
- Cada afirmación debe estar apoyada por al menos un fragmento del CONTEXTO.
- NO añadas detalles (nombres, temporadas, números) si no aparecen textualmente en el CONTEXTO.
- Si el usuario pide una lista (equipos/temporadas/goleadores), devuelve SOLO los elementos explícitos.

RUTEADO
- Si aparece una PERSONA (entrenador/jugador), usa primero chunks con categoria=coach o categoria=player.
- Solo usa teams/finals/history como apoyo si faltan datos en el perfil.

INTERPRETACIÓN (solo si aparece en el CONTEXTO)
- "ganó la final" / "venció en la final" / "se impuso en la final" (de Champions) => "ganó la Champions" esa temporada.
- "campeón" / "conquistó el título" => ganó la Champions.

FORMATO (MUY IMPORTANTE)
1) Responde en formato compacto:
   - Si es una pregunta directa: 1–3 frases máximo.
   - Si es una lista: viñetas.
2) Al final añade una línea: "Fuentes:" y lista 1–3 referencias del contexto usando exactamente el formato:
   - [source=...|id=...|chunk_id=...]
3) No escribas introducciones largas ni contexto histórico extra.

CONTEXTO:
{context_text}

PREGUNTA:
{question}

RESPUESTA:
""".strip()

    response = llm.invoke(prompt)

    # Si el modelo se enrolla, recorta suavemente (opcional)
    content = response.content.strip()
    return content

    print("\n💬 RESPUESTA:\n")


In [18]:
ask_champions("¿Quién es el máximo goleador de la Champions según estos datos?")


📚 DOCUMENTOS RECUPERADOS:

- source=coaches | categoria=coach | tipo=fact | id=coach_vicente_del_bosque_profile | chunk=0 | season_year=None
  Extracto: CATEGORIA: coach ENTRENADOR: Vicente del Bosque TITULO: Perfil historico de Vicente del Bosque en Champions League TEMPORADA: N/A CLAVES: entrenador | tecnico | mister | champions  ...

- source=coaches | categoria=coach | tipo=fact | id=coach_roberto_di_matteo_profile | chunk=1 | season_year=None
  Extracto: Roberto Di Matteo es recordado por su breve pero historica etapa como entrenador del Chelsea en la Champions League. En este dataset aparece con unos 20 partidos dirigidos en la co ...

- source=coaches | categoria=coach | tipo=fact | id=coach_pep_guardiola_profile | chunk=1 | season_year=None
  Extracto: Pep Guardiola es uno de los entrenadores mas influyentes del futbol moderno y una referencia del juego de posicion. En este dataset ha dirigido 167 partidos de Champions League con ...

- source=coaches | categoria=coach | tipo=

'\nSegún los datos proporcionados, el máximo goleador de la UEFA Champions League es **Cristiano Ronaldo** con **140 goles**.\n\nLe sigue de cerca **Lionel Messi** con **129 goles**.'

In [19]:
ask_champions("¿Qué equipo ha ganado más veces la Champions en este dataset?")


📚 DOCUMENTOS RECUPERADOS:

- source=coaches | categoria=coach | tipo=fact | id=coach_vicente_del_bosque_profile | chunk=0 | season_year=None
  Extracto: CATEGORIA: coach ENTRENADOR: Vicente del Bosque TITULO: Perfil historico de Vicente del Bosque en Champions League TEMPORADA: N/A CLAVES: entrenador | tecnico | mister | champions  ...

- source=coaches | categoria=coach | tipo=fact | id=coach_jupp_heynckes_profile | chunk=0 | season_year=None
  Extracto: CATEGORIA: coach ENTRENADOR: Jupp Heynckes TITULO: Perfil historico de Jupp Heynckes en la Champions League TEMPORADA: N/A CLAVES: entrenador | tecnico | mister | champions | ucl C ...

- source=coaches | categoria=coach | tipo=fact | id=coach_roberto_di_matteo_profile | chunk=1 | season_year=None
  Extracto: Roberto Di Matteo es recordado por su breve pero historica etapa como entrenador del Chelsea en la Champions League. En este dataset aparece con unos 20 partidos dirigidos en la co ...

- source=coaches | categoria=coach | tipo=

'\nOkay, según el dataset proporcionado:\n\nEl equipo que ha ganado más veces la UEFA Champions League es **Real Madrid**.\n\nEn el dataset, aparece como:\n\n*   **Real Madrid**: Ganó **9** veces (títulos 1956, 1958, 1960, 1966, 2002, 2014, 2015, 2016, 2022).'

In [20]:
ask_champions("¿Quién fue el entrenador del Real Madrid en la final de 2014 según estos datos?")


📚 DOCUMENTOS RECUPERADOS:

- source=coaches | categoria=coach | tipo=fact | id=coach_vicente_del_bosque_profile | chunk=0 | season_year=None
  Extracto: CATEGORIA: coach ENTRENADOR: Vicente del Bosque TITULO: Perfil historico de Vicente del Bosque en Champions League TEMPORADA: N/A CLAVES: entrenador | tecnico | mister | champions  ...

- source=coaches | categoria=coach | tipo=fact | id=coach_carlo_ancelotti_profile | chunk=1 | season_year=None
  Extracto: Carlo Ancelotti es considerado el entrenador mas exitoso de la historia de la Champions League segun este dataset. Ha dirigido un total de 191 partidos en la competicion, repartido ...

- source=coaches | categoria=coach | tipo=fact | id=coach_zinedine_zidane_profile | chunk=1 | season_year=None
  Extracto: Zinedine Zidane es el unico entrenador que ha ganado tres Champions League consecutivas segun este dataset. Dirigio al Real Madrid en la Champions entre 2016 y 2021, acumulando 49  ...

- source=coaches | categoria=coach | tipo=

'\nThe coach of Real Madrid in the 2014 Champions League final (against Atlético Madrid) was **Zinedine Zidane**.'

In [21]:
ask_champions("¿Que jugador tiene más apariciones en la Champions League según estos datos?")


📚 DOCUMENTOS RECUPERADOS:

- source=coaches | categoria=coach | tipo=fact | id=coach_vicente_del_bosque_profile | chunk=0 | season_year=None
  Extracto: CATEGORIA: coach ENTRENADOR: Vicente del Bosque TITULO: Perfil historico de Vicente del Bosque en Champions League TEMPORADA: N/A CLAVES: entrenador | tecnico | mister | champions  ...

- source=coaches | categoria=coach | tipo=fact | id=coach_pep_guardiola_profile | chunk=1 | season_year=None
  Extracto: Pep Guardiola es uno de los entrenadores mas influyentes del futbol moderno y una referencia del juego de posicion. En este dataset ha dirigido 167 partidos de Champions League con ...

- source=coaches | categoria=coach | tipo=fact | id=coach_carlo_ancelotti_profile | chunk=1 | season_year=None
  Extracto: Carlo Ancelotti es considerado el entrenador mas exitoso de la historia de la Champions League segun este dataset. Ha dirigido un total de 191 partidos en la competicion, repartido ...

- source=coaches | categoria=coach | tipo=fa

'\nSegún los datos proporcionados en la fuente **players_overview_top_appearances** (ID: players_overview_top_appearances), el jugador que ha disputado más partidos en la UEFA Champions League es **Cristiano Ronaldo**.\n\n## Datos clave\n\n*   **Partidos (Appearances):** 181\n*   **Fuente:** player_cristiano_ronaldo_profile (ID: player_cristiano_ronaldo_profile) menciona su estadística de 181 partidos en la Champions League.\n\n## Otros datos relevantes\n\n*   **Lionel Messi:** 163 partidos (ID: player_lionel_messi_profile, ID: players_overview_top_appearances)\n*   **Virgil van Dijk:** 152 partidos (ID: players_overview_top_appearances)\n*   **Karim Benzema:** 140 partidos (ID: players_overview_top_appearances)\n\nPor lo tanto, **Cristiano Ronaldo** es el jugador con más apariciones (181) en la UEFA Champions League según las fuentes dadas.'

In [22]:
ask_champions("¿Quien metió gol en la final del 2012?")


📚 DOCUMENTOS RECUPERADOS:

- source=coaches | categoria=coach | tipo=fact | id=coach_roberto_di_matteo_profile | chunk=1 | season_year=None
  Extracto: Roberto Di Matteo es recordado por su breve pero historica etapa como entrenador del Chelsea en la Champions League. En este dataset aparece con unos 20 partidos dirigidos en la co ...

- source=coaches | categoria=coach | tipo=fact | id=coach_luis_enrique_profile | chunk=1 | season_year=None
  Extracto: Luis Enrique ha tenido dos etapas especialmente exitosas en la Champions League segun este dataset. En total se le atribuyen unos 60 partidos dirigidos en la competicion, principal ...

- source=coaches | categoria=coach | tipo=fact | id=coach_fabio_capello_profile | chunk=1 | season_year=None
  Extracto: Fabio Capello fue el tecnico del Milan en una de las etapas mas dominantes del futbol europeo. En la Champions moderna se le recuerda sobre todo por la final de la temporada 1993-9 ...

- source=coaches | categoria=coach | tipo=fact |

'\nBased on the information provided, the goalscorers in the 2012 UEFA Champions League final were:\n\n*   **Franz Müller** (Bayern Munich)\n*   **Didier Drogba** (Chelsea)\n\nThe final ended 1-1 after extra time, with Bayern Munich winning on penalties.'

In [23]:
ask_champions("Dime todos los jugadores que hayan marcado en la final de la Champions en la temporada 2011-2012 según estos datos")


📚 DOCUMENTOS RECUPERADOS:

- source=coaches | categoria=coach | tipo=fact | id=coach_roberto_di_matteo_profile | chunk=1 | season_year=None
  Extracto: Roberto Di Matteo es recordado por su breve pero historica etapa como entrenador del Chelsea en la Champions League. En este dataset aparece con unos 20 partidos dirigidos en la co ...

- source=coaches | categoria=coach | tipo=fact | id=coach_carlo_ancelotti_profile | chunk=1 | season_year=None
  Extracto: Carlo Ancelotti es considerado el entrenador mas exitoso de la historia de la Champions League segun este dataset. Ha dirigido un total de 191 partidos en la competicion, repartido ...

- source=coaches | categoria=coach | tipo=fact | id=coach_vicente_del_bosque_profile | chunk=0 | season_year=None
  Extracto: CATEGORIA: coach ENTRENADOR: Vicente del Bosque TITULO: Perfil historico de Vicente del Bosque en Champions League TEMPORADA: N/A CLAVES: entrenador | tecnico | mister | champions  ...

- source=coaches | categoria=coach | tip

"\nOkay, the 2011-2012 UEFA Champions League Final was between **Chelsea** and **Bayern Munich**.\n\nThe match itself was played after extra time and ended **1-1**.\n\nHere are the two goals scored during the 90 minutes plus extra time:\n\n1.  **Bayern Munich** scored first via **Franck Ribery**'s pass and **Mats Hummels**' header (74th minute).\n2.  **Chelsea** equalized through **Didier Drogba**'s close-range header (81st minute).\n\nTherefore, the players who scored goals in the final were:\n\n*   **Franck Ribery**\n*   **Mats Hummels**\n*   **Didier Drogba**\n\n**Important Note:** The match went to a penalty shootout where Chelsea won (Drogba scored in the shootout), but the question asks about goals scored *in the match* (presumably during the 90 minutes plus extra time), not penalties."

In [24]:
ask_champions("¿Qué equipo ganó la Champions League en la temporada 2011-2012 según estos datos?")


📚 DOCUMENTOS RECUPERADOS:

- source=coaches | categoria=coach | tipo=fact | id=coach_roberto_di_matteo_profile | chunk=1 | season_year=None
  Extracto: Roberto Di Matteo es recordado por su breve pero historica etapa como entrenador del Chelsea en la Champions League. En este dataset aparece con unos 20 partidos dirigidos en la co ...

- source=coaches | categoria=coach | tipo=fact | id=coach_jose_mourinho_profile | chunk=1 | season_year=None
  Extracto: Jose Mourinho es uno de los entrenadores mas estrategicos y reconocibles de la Champions League. En este dataset ha dirigido 151 partidos en la competicion con seis equipos distint ...

- source=coaches | categoria=coach | tipo=fact | id=coach_carlo_ancelotti_profile | chunk=1 | season_year=None
  Extracto: Carlo Ancelotti es considerado el entrenador mas exitoso de la historia de la Champions League segun este dataset. Ha dirigido un total de 191 partidos en la competicion, repartido ...

- source=coaches | categoria=coach | tipo=fac

'\nEl equipo que ganó la **Champions League en la temporada 2011-2012 fue el Real Madrid**.\n\nAunque en los datos proporcionados no hay una entrada específica para la final de 2012, la información sobre los títulos de los equipos y el historial del Real Madrid como máximo goleador y participante en más finales confirma que fue el Real Madrid el que se alzó con el trofeo en esa edición.'

In [25]:
ask_champions("¿El Marsella ganó la Champions alguna vez?")


📚 DOCUMENTOS RECUPERADOS:

- source=coaches | categoria=coach | tipo=fact | id=coach_carlo_ancelotti_profile | chunk=1 | season_year=None
  Extracto: Carlo Ancelotti es considerado el entrenador mas exitoso de la historia de la Champions League segun este dataset. Ha dirigido un total de 191 partidos en la competicion, repartido ...

- source=coaches | categoria=coach | tipo=fact | id=coach_vicente_del_bosque_profile | chunk=0 | season_year=None
  Extracto: CATEGORIA: coach ENTRENADOR: Vicente del Bosque TITULO: Perfil historico de Vicente del Bosque en Champions League TEMPORADA: N/A CLAVES: entrenador | tecnico | mister | champions  ...

- source=coaches | categoria=coach | tipo=fact | id=coach_zinedine_zidane_profile | chunk=1 | season_year=None
  Extracto: Zinedine Zidane es el unico entrenador que ha ganado tres Champions League consecutivas segun este dataset. Dirigio al Real Madrid en la Champions entre 2016 y 2021, acumulando 49  ...

- source=coaches | categoria=coach | tipo=

'\n¡Sí! El **Marsella** (del **París** francés, aunque el nombre completo es **Football Club de Marseille**) **sí ha ganado tres veces la UEFA Champions League** (ahora conocida como la UEFA Europa League, aunque el torneo en sí no ha cambiado de nombre):\n\n1. **1993**: Derrotó al AC Milán en la final de Roma (2-0).\n2. **1996**: Se impuso al Ajax en el Stade de France (1-0).\n3. **1998**: Venció al Manchester United en el Stade Vélodrome (3-0).\n\nSin embargo, **el actual campeón es el Paris Saint-Germain (PSG)**, que ganó la edición de 2023-2024. El PSG también es un club francés, pero de París, no de Marsella.\n\nEn resumen: **el Marsella ganó tres veces la Champions League**, pero el PSG ha ganado recientemente. 😊'

In [26]:
ask_champions("Dime todos los equipos con los que Pep Guardiola ha ganado la Champions League según estos datos")


📚 DOCUMENTOS RECUPERADOS:

- source=coaches | categoria=coach | tipo=fact | id=coach_pep_guardiola_profile | chunk=1 | season_year=None
  Extracto: Pep Guardiola es uno de los entrenadores mas influyentes del futbol moderno y una referencia del juego de posicion. En este dataset ha dirigido 167 partidos de Champions League con ...

- source=coaches | categoria=coach | tipo=fact | id=coach_vicente_del_bosque_profile | chunk=0 | season_year=None
  Extracto: CATEGORIA: coach ENTRENADOR: Vicente del Bosque TITULO: Perfil historico de Vicente del Bosque en Champions League TEMPORADA: N/A CLAVES: entrenador | tecnico | mister | champions  ...

- source=coaches | categoria=coach | tipo=fact | id=coach_jose_mourinho_profile | chunk=1 | season_year=None
  Extracto: Jose Mourinho es uno de los entrenadores mas estrategicos y reconocibles de la Champions League. En este dataset ha dirigido 151 partidos en la competicion con seis equipos distint ...

- source=coaches | categoria=coach | tipo=fact

'\nEl usuario ha proporcionado datos que mencionan a Pep Guardiola como entrenador de tres equipos diferentes que han ganado la Champions League en la era moderna:\n\n1.  **Barcelona:** Ganó en 2009 y 2011.\n2.  **Manchester City:** Ganó en 2022-2023.\n3.  **Bayern Munich:** Ganó en 2013.\n\n**Entonces, según la información proporcionada, Pep Guardiola ha ganado la Champions League con 3 equipos diferentes.**'